In [1]:
from pymongo import MongoClient
import datetime

client = MongoClient('mongodb://localhost:27017/')
db = client['training_db']
collection = db['employees']

class Employee_Management:
    def insert(self):
        try:
            collection.create_index([("email", 1)], unique=True)
        except Exception:
            pass
    
        while True:
            print("\nInsert as")
            print("1. Single Insert")
            print("2. Bulk Insert")
            print("3. Exit")
            ch = int(input("enter the choice: "))
    
            if ch == 1:
                try:
                    emp_id = int(input("emp_id: "))
                    name = input("name: ")
                    email = input("email: ")
                    department = input("department: ")
                    salary = float(input("salary: "))
                    join_date = datetime.datetime.strptime(input("join_date (YYYY-MM-DD): "), "%Y-%m-%d")
    
                    if collection.count_documents({"email": email}, limit=1):
                        print("duplicate email")
                    else:
                        collection.insert_one({
                            "emp_id": emp_id,
                            "name": name,
                            "email": email,
                            "department": department,
                            "salary": salary,
                            "join_date": join_date,
                            "created_at": datetime.datetime.utcnow(),
                            "updated_at": datetime.datetime.utcnow()
                        })
                        print("Inserted")
                except Exception as e:
                    print("Error inserting document:", e)
    
            elif ch == 2:
                try:
                    n = int(input("how many employees to insert: "))
                    docs = []
                    seen = set()
                    for i in range(n):
                        emp_id = int(input("emp_id: "))
                        name = input("name: ")
                        email = input("email: ")
                        department = input("department: ")
                        salary = float(input("salary: "))
                        join_date = datetime.datetime.strptime(input("join_date (YYYY-MM-DD): "), "%Y-%m-%d")
    
                        if email in seen:
                            print("duplicate email")
                            continue
                        if collection.count_documents({"email": email}, limit=1):
                            print("duplicate email")
                            continue
    
                        docs.append({
                            "emp_id": emp_id,
                            "name": name,
                            "email": email,
                            "department": department,
                            "salary": salary,
                            "join_date": join_date,
                            "created_at": datetime.datetime.utcnow(),
                            "updated_at": datetime.datetime.utcnow()
                        })
                        seen.add(email)
    
                    if docs:
                        collection.insert_many(docs)
                        print("Inserted")
                    else:
                        print("no documents to insert")
                except Exception as e:
                    print("Error inserting documents:", e)
    
            elif ch == 3:
                break
    
    def read_doc(self):
        try:
            for doc in collection.find():
                print(doc)
        except Exception as e1:
            print("Error printing document:", e1)
            
    def search(self):
        while True:
            print("Search as")
            print("1. Search by department")
            print("2. Search by salary range")
            print("3. Search by name pattern")
            print("4. Advance search by multiple criteria")
            print("5. Exit")
            ch1 = int(input("enter the choice: "))
            if ch1 == 1:
                try:
                    dept = input("enter the department: ")
                    for doc in collection.find({"department": dept}):
                        print(doc)
                except Exception as e:
                    print("Error fetching the document:", e)
            elif ch1 == 2:
                try:
                    lower = float(input("enter the lower salary range: "))
                    upper = float(input("enter the upper salary range: "))
                    for doc in collection.find({"salary": {"$gte": lower, "$lte": upper}}):
                        print(doc)
                except Exception as e:
                    print("Error fetching the document:", e)
            elif ch1 == 3:
                try:
                    pat = input("enter the name to search: ")
                    for doc in collection.find({"name": {"$regex": pat}}):
                        print(doc)
                except Exception as e:
                    print("Error fetching the document:", e)
            elif ch1 == 4:
                try:
                    salary = float(input("enter the salary to search: "))
                    dept = input("enter the dept to search: ")
                    for doc in collection.find({"$and": [{"salary": {"$eq": salary}}, {"department": {"$eq": dept}}]}):
                        print(doc)
                except Exception as e:
                    print("Error fetching the document:", e)
            elif ch1 == 5:
                break
                    
    def read_by_page(self):
        try:
            page = int(input("page number (1..): "))
            page_size = int(input("page size: "))
            sort_field = input("sort field (default _id): ") or "_id"
            sort_order = int(input("for asc put 1 and for desc put -1: "))

            skip = (page - 1) * page_size

            cursor = (collection.find()
                      .sort(sort_field, sort_order)
                      .skip(skip)
                      .limit(page_size))

            for doc in cursor:
                print(doc)
        except Exception as e:
            print("Error fetching the document:", e)


    def update_doc(self):
        while True:
            print("Update as")
            print("1. Update single field")
            print("2. Update Multiple Fields")
            print("3. Update by ID")
            print("4. Update by Criteria")
            print("5. Exit")
            ch2 = int(input("enter the choice: "))
            if ch2 == 1:
                try:
                    emp_id = int(input("enter the id to be updated: "))
                    update_field = input("enter the field name to be updated: ")
                    to_be_updated = input("enter the new update: ")
                    result = collection.update_one({"emp_id": emp_id}, {"$set": {update_field: to_be_updated, "updated_at": datetime.datetime.utcnow()}})
                    print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")
                except Exception as e:
                    print("Error updating document:", e)
            elif ch2 == 2:
                try:
                    emp_id = int(input("enter the id to be updated: "))
                    n = int(input("enter the number of updates to be done: "))
                    updates = {}
                    for i in range(n):
                        update_field = input("enter the field to be updated: ")
                        to_be_updated = input("enter the value to be updated: ")
                        updates[update_field] = to_be_updated
                    updates["updated_at"] = datetime.datetime.utcnow()
                    result = collection.update_many({"emp_id": emp_id}, {"$set": updates})
                    print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")
                except Exception as e:
                    print("Error updating document:", e)
            elif ch2 == 3:
                try:
                    emp_id = int(input("enter the id to be updated: "))
                    update_field = input("enter the field name to be updated: ")
                    to_be_updated = input("enter the new update: ")
                    result = collection.update_one({"emp_id": emp_id}, {"$set": {update_field: to_be_updated, "updated_at": datetime.datetime.utcnow()}})
                    print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")
                except Exception as e:
                    print("Error updating document:", e)
            elif ch2 == 4:
                try:
                    field = input("enter the field name to be updated: ")
                    value = input("enter the new value: ")
                    criteria_field = input("enter the criteria field: ")
                    criteria_value = input("enter the criteria value: ")
                    result = collection.update_many({criteria_field: criteria_value}, {"$set": {field: value, "updated_at": datetime.datetime.utcnow()}})
                    print(f"Matched: {result.matched_count}, Modified: {result.modified_count}")
                except Exception as e:
                    print("Error updating document:", e)
            elif ch2 == 5:
                print("exiting")
                break

In [2]:
if __name__ == "__main__":
    em = Employee_Management()
    while True:
        print("\nMenu:")
        print("1. Insert Employees")
        print("2. List All Employees")
        print("3. Search Employees")
        print("4. Paginate Employees")
        print("5. Update Employees")
        print("6. Exit")
        choice = int(input("Enter your choice: "))
        if choice == 1:
            em.insert()
        elif choice == 2:
            em.read_doc()
        elif choice == 3:
            em.search()
        elif choice == 4:
            em.read_by_page()
        elif choice == 5:
            em.update_doc()
        elif choice == 6:
            print("Exiting...")
            break
        else:
            print("Invalid choice")


Menu:
1. Insert Employees
2. List All Employees
3. Search Employees
4. Paginate Employees
5. Update Employees
6. Exit


Enter your choice:  1



Insert as
1. Single Insert
2. Bulk Insert
3. Exit


enter the choice:  1
emp_id:  1002
name:  Gautami
email:  gauatmi@gmail.com
department:  IT
salary:  50000
join_date (YYYY-MM-DD):  2025-09-08


C:\Users\Gautami\AppData\Local\Temp\ipykernel_2052\2518242257.py:41: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.datetime.utcnow(),
C:\Users\Gautami\AppData\Local\Temp\ipykernel_2052\2518242257.py:42: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "updated_at": datetime.datetime.utcnow()


...

Matched: 1, Modified: 1
Update as
1. Update single field
2. Update Multiple Fields
3. Update by ID
4. Update by Criteria
5. Exit


enter the choice:  5


exiting

Menu:
1. Insert Employees
2. List All Employees
3. Search Employees
4. Paginate Employees
5. Update Employees
6. Exit


Enter your choice:  6


Exiting...
